# 類似画像の検出をしよう

## 簡単な形状認識 - Average Hash

最初に、画像の形状を手軽に検出する方法を紹介する。Average Hashを利用すると、画像の類似地を計算することができる。  
Average Hashは、画像を比較可能なハッシュ値で表す。ハッシュ関数MD5やSHA256などを使うと、様々なデータの値を要約したハッシュ値を得ることができる。そして、これらのハッシュ値は同一データを検出するのに非常に役立つ。  
しかし、大きな意味で画像が同一かどうかを検出するのに、MD5やSHA256などのハッシュ関数を利用することはできない。完全に同一のバイナリを探すならば、こちらのハッシュ関数を利用することができるが、画像データは画像解像度をリサイズしたり、色調補正したり、JPEG/PNGと圧縮形式を変更したりと、様々な変更が加えられ、完全に同一ではないことも多い。  
写真を多少明るく補正してあったとしても、同一の画像として検索できた方が便利な場面が多くあり、その時に役立つのが、類似度の計算に使えるAverage Hashである。  
具体的な方法だが、以下のような手順で行う。  
1. 画像のサイズを8*8に縮小
2. 色をグレイスケールに変換
3. 画像の各ピクセルの平均値を計算
4. 各ピクセルの濃淡が平均より大きければ1、平均以下なら0とする  
以上のような簡単な手順で、画像の形状を表す64ビットのハッシュ値が得られる。この方法であれば、64ビットを比較するだけなので、高速に類似画像を検索することができる。

In [3]:
from PIL import Image
import numpy as np

In [7]:
# 画像データをAverage Hashに変換
def average_hash(fname, size = 16):
    img = Image.open(fname)
    img = img.convert('L') # グレースケールに変換
    img = img.resize((size, size), Image.ANTIALIAS) # アンチエイリアス処理を施した後リサイズ
    pixel_data = img.getdata() # ピクセルデータを得る
    pixels = np.array(pixel_data) # Numpyの配列に変換
    pixels = pixels.reshape((size, size)) # 二次元の配列に変換
    avg = pixels.mean() # 算術平均を計算
    diff = 1 * (pixels > avg) # 平均より大きければ値を1、平均以下で0に変換
    return diff

In [8]:
# 二進数とみなしてハッシュ値に変換
def np2hash(n):
    bhash = []
    for nl in ahash.tolist():
        sl = [str(i) for i in nl]
        s2 ="".join(sl)
        i = int(s2, 2) # 二進数を整数に
        bhash.append("%04x" % i)
    return "".join(bhash)

In [9]:
# Average Hashを表示
ahash = average_hash('tower.jpg')
print(ahash)
print(np2hash(ahash))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0]
 [0 1 0 1 1 1 1 1 1 1 1 1 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0]
 [1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0]
 [1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1]]
000001000180018003c003c003c087e07ff05ff07ff2cffa9ffe3ffe3ffe1fff


### たくさんの画像から類似する画像を検索する

Average Hashがわかったところで、たくさんの画像の中からこの画像に類似するものを選び出すプログラムを作っていく。ここでは、サンプル画像として、カリフォルニア工科大学が機械学習のために配布している様々な画像データセット「Computational Vision at CALTECH」を利用してみる。

In [27]:
from PIL import Image
import numpy as np
import os, re

In [28]:
# ファイルパスの指定
search_dir = "./image/101_ObjectCategories"
cache_dir = "./image/cache_avhash"

In [29]:
if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)

In [30]:
# 画像データをAverage hashに変換
def average_hash(fname, size=16):
    fname2 = fname[len(search_dir):]
    # 画像をキャッシュしておく
    cache_file = cache_dir + "/" + fname2.replace('/', '_') + ".csv"
    if not os.path.exists(cache_file): # ハッシュを作成
        img = Image.open(fname)
        img = img.convert('L').resize((size, size), Image.ANTIALIAS)
        pixels = np.array(img.getdata()).reshape((size, size))
        avg = pixels.mean()
        px = 1 * (pixels > avg)
        np.savetxt(cache_file, px, fmt="%.0f", delimiter=",")
    else: # すでにキャッシュがあればファイルから読み込み
        px = np.loadtxt(cache_file, delimiter=",")
    return px

In [31]:
# 簡単にハミング距離を求める
def hamming_dist(a, b):
    aa = a.reshape(1, -1) # 1次元の配列に変換
    ab = b.reshape(1, -1)
    dist = (aa != ab).sum()
    return dist

In [32]:
# 全てのディレクトリを列挙
def enum_all_files(path):
    for root, dirs, files in os.walk(path):
        for f in files:
            fname = os.path.join(root, f)
            if re.search(r'\.(jpg|jpeg|png)$', fname):
                yield fname

In [33]:
# 画像を検索
def find_image(fname, rate):
    src = average_has(fname)
    for fname in enum_all_files(search_dir):
        dst = average_hash(fname)
        diff_r = hamming_dist(src, dst)/256
        # print("[check] ", fname)
        if diff_r < rate:
            yield (diff_r, fname)

In [34]:
# 検索
srcfile = search_dir + "/chair/image_0016.jpg"
html = ""
sim = list(find_image(srcfile, 0.25))
sim = sorted(sim, key=lambda x:x[0])
for r, f in sim:
    print(r, ">", f)
    s = '<div style="float:left;"><h3>[差異:' + str(r) + '-' + \
        os.path.basename(f) + ']</h3>' + \
    '<p><a href="' + f + '"><img src="' + f + '" width = 400>' + \
    '</a></p></div>'
    html += s
# HTMLを出力
html = """<html><body><h3>元画像</h3><p>
<img src='{0}' width=400></p>{1}
</body></html>""".format(srcfile, html)
with open("./avhash-search-output.html", "w", encoding="utf-8") as f:
    f.write(html)
print("ok")

0.0 > ./image/101_ObjectCategories/chair/image_0016.jpg
0.22265625 > ./image/101_ObjectCategories/airplanes/image_0129.jpg
0.2265625 > ./image/101_ObjectCategories/chair/image_0031.jpg
0.2265625 > ./image/101_ObjectCategories/stop_sign/image_0019.jpg
0.234375 > ./image/101_ObjectCategories/umbrella/image_0009.jpg
0.23828125 > ./image/101_ObjectCategories/airplanes/image_0124.jpg
0.24609375 > ./image/101_ObjectCategories/dragonfly/image_0001.jpg
0.24609375 > ./image/101_ObjectCategories/chair/image_0001.jpg
ok


ハミング距離とは、等しい文字数を持つ3つの文字列の中で、対応する一にあることなった文字の個数のこと。ここでは、画像一つを256字のハッシュ値で表しているため、何文字異なるかを調べて、それを画像の差異として表している。